# DEMO: Stokes/Correlation Conversion for Visibility Data

[Colab Link](https://colab.research.google.com/github/casangi/astroviper/blob/main/docs/core_tutorials/imaging/demo_stokes_correlation_conversion.ipynb)

This notebook demonstrates how to convert visibility data between feed basis (correlation products) and Stokes basis using the `corr_to_stokes` and `stokes_to_corr` functions.

**Key Concepts:**
- **Correlation products** represent the raw output from interferometer feeds (e.g., XX, XY, YX, YY for linear feeds or RR, RL, LR, LL for circular feeds)
- **Stokes parameters** (I, Q, U, V) represent physical polarization properties:
  - I: Total intensity
  - Q: Linear polarization (horizontal vs vertical)
  - U: Linear polarization (±45°)
  - V: Circular polarization

**This tutorial covers:**
1. Linear polarization conversion (XX, XY, YX, YY ↔ I, Q, U, V)
2. Circular polarization conversion (RR, RL, LR, LL ↔ I, Q, U, V)
3. Round-trip conversion verification
4. Working with multidimensional visibility arrays
5. xarray DataArray compatibility

## Install AstroVIPER

Skip this cell if you don't want to install the latest version of AstroVIPER.

In [ ]:
from importlib.metadata import version
import os

try:
    os.system("pip install --upgrade astroviper")

    import astroviper

    print("Using astroviper version", version("astroviper"))

except ImportError as exc:
    print(f"Could not import astroviper: {exc}")

## Imports

In [ ]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
from astroviper.core.imaging.imaging_utils.corr_to_stokes import corr_to_stokes, stokes_to_corr

# Set random seed for reproducibility
np.random.seed(42)

## 1. Linear Polarization: Correlation Products ↔ Stokes Parameters

Linear feeds produce correlation products in the order: **[XX, XY, YX, YY]**

These convert to Stokes parameters **[I, Q, U, V]** using:
- I = XX + YY (total intensity)
- Q = XX - YY (linear polarization)
- U = XY + YX (linear polarization at 45°)
- V = i(YX - XY) (circular polarization)

### Generate synthetic linear correlation visibility data

We'll create a simple example with known properties to verify the conversion.

In [ ]:
# Create synthetic visibility data for a source with:
# - Total intensity I = 10.0 Jy
# - Linear polarization Q = 2.0 Jy (slightly more power in XX than YY)
# - No linear polarization at 45° (U = 0)
# - No circular polarization (V = 0)

# From the inverse formulas:
# XX = (I + Q)/2 = (10 + 2)/2 = 6.0
# YY = (I - Q)/2 = (10 - 2)/2 = 4.0
# XY = (U + iV)/2 = (0 + 0)/2 = 0.0
# YX = (U - iV)/2 = (0 - 0)/2 = 0.0

linear_corr = np.array([6.0 + 0.0j, 0.0 + 0.0j, 0.0 + 0.0j, 4.0 + 0.0j])

print("Input linear correlation products [XX, XY, YX, YY]:")
print(linear_corr)
print(f"\nShape: {linear_corr.shape}")

### Convert correlation products to Stokes parameters

In [ ]:
# Convert to Stokes parameters
stokes_linear = corr_to_stokes(linear_corr, corr_type='linear')

print("Output Stokes parameters [I, Q, U, V]:")
print(stokes_linear)
print(f"\nExpected: [10.0, 2.0, 0.0, 0.0]")
print(f"Match: {np.allclose(stokes_linear, [10.0, 2.0, 0.0, 0.0])}")

### Convert Stokes parameters back to correlation products

In [ ]:
# Convert back to correlation products
linear_corr_roundtrip = stokes_to_corr(stokes_linear, corr_type='linear')

print("Round-trip correlation products [XX, XY, YX, YY]:")
print(linear_corr_roundtrip)
print(f"\nOriginal:")
print(linear_corr)
print(f"\nRound-trip successful: {np.allclose(linear_corr, linear_corr_roundtrip)}")

## 2. Circular Polarization: Correlation Products ↔ Stokes Parameters

Circular feeds produce correlation products in the order: **[RR, RL, LR, LL]**

These convert to Stokes parameters **[I, Q, U, V]** using:
- I = RR + LL (total intensity)
- Q = RL + LR (linear polarization)
- U = i(LR - RL) (linear polarization at 45°)
- V = RR - LL (circular polarization)

### Generate synthetic circular correlation visibility data

In [ ]:
# Create synthetic visibility data for a source with:
# - Total intensity I = 8.0 Jy
# - No linear polarization (Q = 0, U = 0)
# - Circular polarization V = 2.0 Jy (more right-hand than left-hand)

# From the inverse formulas:
# RR = (I + V)/2 = (8 + 2)/2 = 5.0
# LL = (I - V)/2 = (8 - 2)/2 = 3.0
# RL = (Q + iU)/2 = (0 + 0)/2 = 0.0
# LR = (Q - iU)/2 = (0 - 0)/2 = 0.0

circular_corr = np.array([5.0 + 0.0j, 0.0 + 0.0j, 0.0 + 0.0j, 3.0 + 0.0j])

print("Input circular correlation products [RR, RL, LR, LL]:")
print(circular_corr)
print(f"\nShape: {circular_corr.shape}")

### Convert correlation products to Stokes parameters

In [ ]:
# Convert to Stokes parameters
stokes_circular = corr_to_stokes(circular_corr, corr_type='circular')

print("Output Stokes parameters [I, Q, U, V]:")
print(stokes_circular)
print(f"\nExpected: [8.0, 0.0, 0.0, 2.0]")
print(f"Match: {np.allclose(stokes_circular, [8.0, 0.0, 0.0, 2.0])}")

### Convert Stokes parameters back to correlation products

In [ ]:
# Convert back to correlation products
circular_corr_roundtrip = stokes_to_corr(stokes_circular, corr_type='circular')

print("Round-trip correlation products [RR, RL, LR, LL]:")
print(circular_corr_roundtrip)
print(f"\nOriginal:")
print(circular_corr)
print(f"\nRound-trip successful: {np.allclose(circular_corr, circular_corr_roundtrip)}")

## 3. Multidimensional Visibility Arrays

Real visibility data has multiple dimensions: time, baseline, frequency, and polarization.
The conversion functions work on any shape as long as the polarization dimension is last.

In [ ]:
# Create realistic multi-dimensional visibility data
# Shape: (time=10, baseline=15, frequency=32, polarization=4)
n_time = 10
n_baseline = 15
n_freq = 32
n_pol = 4

# Generate random complex visibility data
# In reality, this would come from your measurement set
vis_corr = np.random.randn(n_time, n_baseline, n_freq, n_pol) + \
           1j * np.random.randn(n_time, n_baseline, n_freq, n_pol)

print(f"Input visibility shape: {vis_corr.shape}")
print(f"Dimensions: (time={n_time}, baseline={n_baseline}, frequency={n_freq}, polarization={n_pol})")

In [ ]:
# Convert to Stokes
vis_stokes = corr_to_stokes(vis_corr, corr_type='linear')

print(f"\nStokes visibility shape: {vis_stokes.shape}")
print("The shape is preserved, only the last dimension is transformed.")

In [ ]:
# Convert back to correlation products
vis_corr_roundtrip = stokes_to_corr(vis_stokes, corr_type='linear')

print(f"\nRound-trip visibility shape: {vis_corr_roundtrip.shape}")
print(f"Round-trip successful: {np.allclose(vis_corr, vis_corr_roundtrip)}")
print(f"Maximum absolute error: {np.max(np.abs(vis_corr - vis_corr_roundtrip))}")

## 4. xarray DataArray Compatibility

The conversion functions also accept xarray DataArrays.
The output is always a numpy array.

In [ ]:
# Create an xarray DataArray with labeled dimensions and coordinates
vis_xarray = xr.DataArray(
    vis_corr,
    dims=['time', 'baseline_id', 'frequency', 'polarization'],
    coords={
        'time': np.arange(n_time),
        'baseline_id': np.arange(n_baseline),
        'frequency': np.linspace(1.4e9, 1.5e9, n_freq),  # 1.4-1.5 GHz
        'polarization': ['XX', 'XY', 'YX', 'YY']
    }
)

print("Input xarray DataArray:")
print(vis_xarray)
print(f"\nData type: {type(vis_xarray)}")

In [ ]:
# Convert xarray DataArray to Stokes
vis_stokes_from_xarray = corr_to_stokes(vis_xarray, corr_type='linear')

print(f"\nOutput type: {type(vis_stokes_from_xarray)}")
print(f"Output shape: {vis_stokes_from_xarray.shape}")
print("\nNote: The output is a numpy array, not an xarray DataArray.")

In [ ]:
# Verify the xarray conversion matches the numpy conversion
print(f"\nxarray and numpy conversions match: {np.allclose(vis_stokes, vis_stokes_from_xarray)}")